# Update a Session Record

This notebook updates a record in a specific table by session_id.

**Use this when:**
- You need to correct data in a specific table
- You want to update specific fields for a session


In [ ]:
import sys
from pathlib import Path
from typing import Dict, Any

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

from src.adapters.database.database_handler import DatabaseHandler


## Configuration

Set `USE_DYNAMODB = True` to use DynamoDB, or `False` to use CSV files.


In [ ]:
# Configuration
USE_DYNAMODB = True  # Set to False to use CSV instead


In [ ]:
def update_session_record(
    table_name: str,
    session_id: int,
    update_data: Dict[str, Any],
    db_write_allowed: bool = True
) -> bool:
    """
    Update a record in a specific table by session_id.
    
    Args:
        table_name: Name of the table to update
        session_id: Session ID of the record to update
        update_data: Dictionary of fields to update
        db_write_allowed: If True, use DynamoDB; if False, use CSV
        
    Returns:
        True if update was successful, False otherwise
    """
    print("=" * 60)
    print(f"Updating Session Record")
    print("=" * 60)
    print(f"Table: {table_name}")
    print(f"Session ID: {session_id}")
    print(f"Backend: {'DynamoDB' if db_write_allowed else 'CSV'}")
    print("=" * 60)
    
    db_handler = DatabaseHandler(db_write_allowed=db_write_allowed)
    
    try:
        # Check if record exists
        print(f"\n[1] Checking if record exists in {table_name}...")
        table_data = db_handler.load_table(table_name)
        
        if table_data.empty:
            print(f"[ERROR] Table '{table_name}' is empty")
            db_handler.close()
            return False
        
        # Find the record
        record = table_data[table_data["id"] == session_id]
        
        if record.empty:
            print(f"[ERROR] Record with session_id={session_id} not found in {table_name}")
            db_handler.close()
            return False
        
        print(f"[OK] Found record in {table_name}")
        print(f"     Current values: {dict(record.iloc[0].head(5))}...")
        
        # Update the record
        print(f"\n[2] Updating record...")
        print(f"     Fields to update: {list(update_data.keys())}")
        
        db_handler.update_record(
            table_name=table_name,
            key_dict={"id": session_id},
            update_dict=update_data
        )
        
        print(f"[OK] Record updated successfully!")
        
        db_handler.close()
        
        print("\n" + "=" * 60)
        print("[SUCCESS] Record updated!")
        print("=" * 60)
        return True
        
    except Exception as e:
        print(f"\n[ERROR] Error updating record: {e}")
        import traceback
        print(f"[ERROR] Traceback: {traceback.format_exc()}")
        db_handler.close()
        return False


In [ ]:
# Example: Update a record
# Uncomment and modify the values below to update a record

# update_session_record(
#     table_name="session_responses",
#     session_id=123456789,
#     update_data={
#         "toxic_score": 0.75,
#         "filter_violations": 2
#     },
#     db_write_allowed=USE_DYNAMODB
# )
